In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import pickle

In [2]:
#use pickle file
SPLIT_POINT = 590540


with open('train_test_590540.pkl', 'rb') as f:
    df_all_dy = pickle.load(f)
    print("df_all load finished")

    
df_train = df_all_dy[:SPLIT_POINT]

df_test = df_all_dy[SPLIT_POINT:].reset_index()
df_test = df_test.drop(['isFraud', 'index'])

df_all load finished


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 267 entries, index to card6__nan
dtypes: float16(187), float32(22), float64(1), int16(1), int32(2), int64(1), uint8(53)
memory usage: 304.7 MB


In [30]:
df_train.head(3)

,index,C1,C10,C11,C12,C13,C14,C2,C3,C4,...,card4__american express,card4__discover,card4__mastercard,card4__visa,card4__nan,card6__charge card,card6__credit,card6__debit,card6__debit or credit,card6__nan
0,0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0,1,0,0,0,0,1,0,0,0
1,1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0,0,1,0,0,0,1,0,0,0
2,2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0


In [29]:
df_test.head(3)

,index,C1,C10,C11,C12,C13,C14,C2,C3,C4,...,card4__american express,card4__discover,card4__mastercard,card4__visa,card4__nan,card6__charge card,card6__credit,card6__debit,card6__debit or credit,card6__nan
590540,0,6.0,0.0,5.0,1.0,115.0,6.0,6.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
590541,1,3.0,0.0,1.0,1.0,12.0,2.0,2.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
590542,2,2.0,0.0,2.0,0.0,22.0,2.0,2.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 590540 to 1097230
Columns: 266 entries, index to card6__nan
dtypes: float16(187), float32(22), int16(1), int32(2), int64(1), uint8(53)
memory usage: 257.6 MB


In [31]:
import xgboost as xgb

clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    random_state=2019,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

%time clf.fit(df_train.drop(['isFraud', 'index'], axis=1), df_train['isFraud'])

XGBoostError: [22:34:10] src/learner.cc:180: XGBoost version not compiled with GPU support.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000018c586579 dmlc::LogMessageFatal::~LogMessageFatal() + 57
  [bt] (1) 2   libxgboost.dylib                    0x000000018c589e70 xgboost::LearnerImpl::ConfigureUpdaters() + 2016
  [bt] (2) 3   libxgboost.dylib                    0x000000018c581971 xgboost::LearnerImpl::Configure(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > > > > const&) + 3633
  [bt] (3) 4   libxgboost.dylib                    0x000000018c5a12eb XGBoosterUpdateOneIter + 139
  [bt] (4) 5   _ctype

In [21]:
#sample_submission = pd.read_csv('data/ieee-fraud-detection/sample_submission.csv')
sample_submission['isFraud'] = clf.predict_proba(df_test.drop(['index']))[:,1]
sample_submission.to_csv('simple_xgboost-1.csv', index=False, columns=['TransactionID', 'isFraud'])

In [22]:
ans = pd.read_csv('simple_xgboost-1.csv')


In [23]:
ans.head(10)

,TransactionID,isFraud
0,3663549,0.001075
1,3663550,0.001328
2,3663551,0.002051
3,3663552,0.000613
4,3663553,0.001981
5,3663554,0.001593
6,3663555,0.004570
7,3663556,0.025996
8,3663557,0.000199
9,3663558,0.004498


In [24]:
ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Data columns (total 2 columns):
TransactionID    506691 non-null int64
isFraud          506691 non-null float64
dtypes: float64(1), int64(1)
memory usage: 7.7 MB


In [19]:
sample_submission.to_csv('simple_xgboost-1.csv', index=False)

In [28]:
df_train['isFraud'].sum()

20663.0

In [ ]:
'''
#li yao lin
##titanic

#han zong da
##编码器、zhen liu

#
##study

#mei zi
##cv

#zhu guo zhen
#封装dt

#bai wen tao
##模糊means、



'''